In [6]:
with open('data/smirnenski.txt') as f:
    content = f.read()

In [22]:
print(content[:10], ' - ', len(content))

Нощта е че  -  21589


In [76]:
from nltk.tokenize import sent_tokenize, TweetTokenizer
from string import punctuation
from os import scandir
tokenizer = TweetTokenizer()

def preprocess_text(content):
    sentences = sent_tokenize(content)
    tokens = []
    for _sent in sentences:
        sent_tokens = tokenizer.tokenize(_sent)
        sent_tokens = [_tok.lower() for _tok in sent_tokens if _tok not in punctuation]
        tokens += sent_tokens
    return tokens

In [103]:
sorted_tokens = preprocess_text(content)

In [104]:
print(len(sorted_tokens), len(set(sorted_tokens)))

3644 1673


In [105]:
from collections import defaultdict

def calc_bigram_freq(sorted_tokens):
    result = defaultdict(lambda: defaultdict(lambda: 0))

    for f, s in zip(sorted_tokens, sorted_tokens[1:]):
        result[f][s] += 1
        
    return result

In [217]:
bigram_freq = calc_bigram_freq([token for token in sorted_tokens if len(token) > 0])

In [218]:
bigram_freq_dict = {k: dict(v) for k, v in bigram_freq.items()}
list(bigram_freq_dict.items())[50:60]

[('води', {'и': 1}),
 ('издигат', {'се': 1}),
 ('бронята', {'на': 1}),
 ('назад', {'а': 1, 'назад': 1}),
 ('учуди', {'зовът': 1}),
 ('синове', {'безумци': 1, 'през': 1}),
 ('развява', {'пред': 1}),
 ('край', {'нас': 1, 'него': 1}),
 ('верига', {'на': 1}),
 ('безумно', {'крещи': 1, 'смелата': 1})]

In [219]:
sorted(bigram_freq_dict.items(), key=lambda x: max(map(lambda y: y[1], x[1].items())))[-4:]

[('на',
  {'акациите': 1,
   'барикадата': 2,
   'бедните': 1,
   'берлин': 1,
   'битката': 1,
   'борби': 1,
   'братята': 1,
   'буйните': 1,
   'бунт': 1,
   'бунта': 1,
   'буря': 1,
   'вековните': 1,
   'вечен': 1,
   'властен': 1,
   'войната': 2,
   'вседневен': 1,
   'вулкана': 1,
   'гърди': 1,
   'далечни': 1,
   'дебнещата': 1,
   'дете': 1,
   'дом': 1,
   'душата': 1,
   'електрическите': 1,
   'жестоката': 1,
   'живота': 1,
   'жълтия': 1,
   'забравата': 1,
   'земната': 1,
   'земните': 1,
   'земния': 1,
   'земята': 1,
   'златний': 1,
   'злокобните': 1,
   'индия': 1,
   'кървав': 2,
   'ледна': 1,
   'лондон': 1,
   'майката': 3,
   'майката-земя': 1,
   'малкия': 1,
   'младите': 1,
   'мойте': 1,
   'моята': 1,
   'мрак': 1,
   'мрачния': 1,
   'музика': 1,
   'мъдрост': 1,
   'нашата': 1,
   'невинност': 1,
   'нежно': 1,
   'нови': 1,
   'нощта': 2,
   'някакъв': 1,
   'обезличени': 1,
   'огнени': 1,
   'огнено': 1,
   'океанските': 1,
   'падаща': 1,
   'п

In [220]:
bigram_prob = {f: {s: freq / sum(map(lambda x: x[1], goto.items())) for s, freq in goto.items()} 
               for f, goto in bigram_freq_dict.items()}

In [221]:
list(bigram_prob.items())[10:20]

[('силует', {'на': 1.0}),
 ('отговарят', {'чрез': 1.0}),
 ('сребросинкави', {'петна': 1.0}),
 ('улицата', {'със': 0.5, 'шумна': 0.5}),
 ('строга', {'на': 1.0}),
 ('хладната', {'си': 1.0}),
 ('озарен', {'от': 0.5, 'с': 0.5}),
 ('стозвучно', {'загърмя': 1.0}),
 ('мощ',
  {'горите': 0.3333333333333333,
   'заканата': 0.3333333333333333,
   'слепци': 0.3333333333333333}),
 ('друго', {'рухват': 1.0})]

In [262]:
from random import random, choice, seed as seed_random, shuffle

def prob_choice(items, key=lambda x: x, seed=None):
    seed_random(seed)

    weights = map(key, items)
    total = sum(weights)
    s = 0

    for i, w in enumerate(weights):
        if s >= random():
            return items[i]
        s += w
    return choice(list(items))
    

def prob_walk(seed_word, mutation, gen_size, bigram_prob, seed=None):
    result = [seed_word]
    for _ in range(gen_size):
        new_word = prob_choice(bigram_prob[result[-1]].items(), key=lambda x: x[1], seed=seed)
        result.append(new_word[0])
        
    return result

In [263]:
generated_tokens = prob_walk('улицата', 1, 20, bigram_prob, 5)

In [264]:
print(" ".join(generated_tokens))

улицата със поглед замечтан навън цареше смъртна пустота и стене глухо прибледнелите уста а ний нов път хусарите сред дима на


In [265]:
generated_tokens = prob_walk('деца', 1, 15, bigram_prob, 0)

In [266]:
print(' '.join(generated_tokens))

деца на черна нерадост възмъжах и колко скръб в сърцата пролетарски свят спартаков път над тях


In [276]:
def find_rhyming(word, tokens):
    shuffle(tokens)
    for n in range(4, 0, -1):
        for token in tokens:
            if token != word and token.endswith(word[-n:]):
                return token
    return choice(tokens)

In [277]:
find_rhyming('улицата', sorted_tokens)

'десницата'

In [279]:
prob_walk('деца', 1, 4, bigram_prob, 0)

['деца', 'на', 'черна', 'нерадост', 'възмъжах']

In [280]:
find_rhyming('възмъжах', sorted_tokens)

'исках'